This notebook is used to demonstrate how to probe the relationship between geophysical properties using spatial queries and interpolation. Some parts of this notebook are rather messy as a result of
needing to use data from two different AEM inversions

Neil Symington

neil.symington@ga.gov.au

In [1]:
%matplotlib inline

In [2]:
import netCDF4
import pandas as pd
import shapely
import numpy as np
import matplotlib.pyplot as plt
import sqlalchemy as db
from sqlite3 import dbapi2 as sqlite
from geophys_utils._netcdf_point_utils import NetCDFPointUtils
from hydrogeol_utils import SNMR_utils, spatial_functions, AEM_utils

In [3]:

# Funcntion for finding the closest AEM survey for a given point
# using the distances. This is a rather hacky function that is
# necesary where two or more AEM surveys may overlap

def return_closest_survey(aem_keys, distances, count):
    # Add the minimum distance to a dictionary with the AEM key
    min_distances = {}
    # Iterate through the keys
    for key in aem_keys.keys():
        # Find the minimum distance
        min_distances[key] = np.min(distances[key][count])
    # Return the key with the smallest corresponding value
    return min(min_distances, key=min_distances.get)        




In [4]:
# First we will import the AEM conductivity point line data
# The AEM data is in a netCDF file created from a asci files using geophys_utils
# https://github.com/GeoscienceAustralia/geophys_utils
# Start with inversions from the Ord-Bonaparte AEM survey
infile = r"D:\GA\OB_AEM\inversions\netcdf\OrdBon2019_ModelExp_cord2dLogKeep.nc"
ob_cond = netCDF4.Dataset(infile)

# Load East Kimberley data
infile  = r"D:\GA\EK_AEM\inversions\netcdf\NBC_ModelExp_cor2d.nc"
ek_cond = netCDF4.Dataset(infile)

# This dictionary is used in the extract_cond_profile function to link the key from
# the indices and distances dictionary to the AM data

aem_keys = {'ek': ek_cond,
            'ob': ob_cond}

In [25]:
# List the variables for the Ord Bonaparte AEM data
print(ob_cond)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: Dataset read from ASEG-GDF file OrdBon2019_ModelExp_cord2dLogKeep.dat
    Conventions: CF-1.6,ACDD-1.3
    featureType: trajectory
    geospatial_vertical_min: -6.44
    geospatial_vertical_max: 183.83
    geospatial_vertical_units: m
    geospatial_vertical_resolution: point
    geospatial_vertical_positive: up
    history: Converted from ASEG-GDF file OrdBon2019_ModelExp_cord2dLogKeep.dat using definitions file OrdBon2019_ModelExp_cord2dLogKeep.dfn
    date_created: 2018-12-03T10:32:35.353808
    geospatial_east_resolution: point
    geospatial_north_resolution: point
    geospatial_east_min: 439425.0
    geospatial_east_max: 519593.2
    geospatial_east_units: m
    geospatial_north_min: 8272446.0
    geospatial_north_max: 8371328.0
    geospatial_north_units: m
    geospatial_bounds: POLYGON((497426.3000 8272446.0000, 462013.2000 8307557.0000, 458094.8000 8312040.0000, 457222.0000 83143

In [6]:
# Here we demonstrate that the surveys are similar enough that the data can be treated together

# Check all layer top depths are the essentially the same for all points in both surveys


# Check coordinate systems are the same

print(ob_cond.variables['transverse_mercator'].spatial_ref == 
      ek_cond.variables['transverse_mercator'].spatial_ref)

# Check that the number of layers are the same and that the layers are less than 1.5 cm difference in
# depth

# Check number of layers
print(ob_cond.variables['layer_top_depth'][:].shape[1] ==
      ek_cond.variables['layer_top_depth'][:].shape[1])

# Check the all layers are the same through all layers 
print(np.max(np.abs(ob_cond.variables['layer_top_depth'][:] -
                     ob_cond.variables['layer_top_depth'][:][0])))

print(np.max(np.abs(ek_cond.variables['layer_top_depth'][:] -
                     ek_cond.variables['layer_top_depth'][:][0])))

# Check the maximum difference in layer depth

print(np.max(np.abs(ob_cond.variables['layer_top_depth'][:][0]
                    - ek_cond.variables['layer_top_depth'][:][0])))


True
True
0.0
0.01499939
0.010009766


In [7]:
# Load the spatial metadata into memory

# Connect to the SNMR database

DB_PATH = r"D:\GA\SNMR\EastKimberley_SNMR_models.sqlite"
        
engine = db.create_engine('sqlite:///' + DB_PATH, module=sqlite)

connection = engine.connect()

# Load the site table identifier and wkt geometry from the spatialite database
# using a sql query
df_sites = pd.read_sql_query("select table_id, mid_x, mid_y from sites;", connection,
                             index_col = 'table_id')


In [8]:
# Create a point utils object for the Ord Bonaparte AEM dataset and find the
# nearest neighbour's distance in metres and netCDF point index

distances = {}
indices = {}

ob_point_util = NetCDFPointUtils(ob_cond)
ek_point_util = NetCDFPointUtils(ek_cond)

SNMR_coords = df_sites[['mid_X', 'mid_Y']].values

# Note that the max distance is not working so we will simply use the nearest
# ten points for each survey. Combine them into the nearest 
distances['ob'], indices['ob'] = ob_point_util.nearest_neighbours(SNMR_coords, 
                           points_required=10)

distances['ek'], indices['ek'] = ek_point_util.nearest_neighbours(SNMR_coords, 
                           points_required=10)


In [ ]:
# The next step is to iterate through the sites, choose which survey to extract
# our points from based on the smallest distance, use an inverse distance 
# weighting scheme to calculate weights and use this to estimate a representative
# conductivity profile for each site

# We will save the profiles in a pandas dataframe with a SNMR site key column

df_conductivity = pd.DataFrame(columns = {'SNMR_id',
                                          'Depth_from',
                                          'Depth_to',
                                          'conductivity'})

# Iteratively extract the conductivity profiles

for i, (index, row) in enumerate(df_sites.iterrows()):
    # Find which survey has the single closest fiducial
    closest_aem_key = return_closest_survey(aem_keys, distances, i)

    
    df_conductivity_profile = AEM_utils.extract_conductivity_profile(aem_keys[closest_aem_key],
                                                   distances[closest_aem_key][i],
                                                   indices[closest_aem_key][i],
                                                   as_dataframe = True)
    # Add the SNMR id to the column
    df_conductivity_profile['SNMR_id'] = i
    
    # Append the site profile to the dataframe
    df_conductivity = df_conductivity.append(df_conductivity_profile, sort = True)
    
df_conductivity.reset_index(inplace=True, drop=True)

Our next step is to extract our best inversion for each site. As we do not yet have a good approachc to reconcile multiple inversions generated on dat acquired using variable acquisition parameters our approach will be to choose the best available using the following criteria

1. Use inversions from free induction decay (FID) data over CPMG or T1
2. Use larger pulse widths over smaller

In [13]:
# Extract the acqusition data

query= 'SELECT * FROM acquisitions'
df_acquisitions = pd.read_sql_query(query, connection, index_col = 'table_id')

# Extract the acquisitions based on criteria

# Our first criteria will be an ordered list for how we preference
# the acquisition type

pulse_sequence_criteria = ['FID', 'T1', 'CPMG']

# Our first criteria is choosing the larger available pulse length

pulse_length_criteria = 'max'

acquisition_ids = SNMR_utils.choose_snmr_site_acquisition(df_acquisitions,
                                               pulse_sequence_criteria,
                                               pulse_length_criteria)


In [14]:
# Extract the inversion table for these acquisitions

cols = ['Depth_from', 'Depth_to', 'Mobile_water_content', 'Bound_water_content',
       'Total_water_content', 'site_id', 'acquisition_id', 'inversion_id']

df_inversions= SNMR_utils.extract_snmr_inversions(acquisition_ids, connection,
                                                  mask_below_doi = True)[cols]

In [ ]:
# Iterate through the acquisitions and interpolate the SNMR intervals onto the AEM intervals
# using an average

# Define the columns that are being interpolated 
parameter_columns = ['Mobile_water_content', 'Bound_water_content', 
                     'Total_water_content']
interval_columns = ['Depth_from','Depth_to']

# Create interpolated parameter columns in the conductivity dataframe
interpolated_columns =[]
for p in parameter_columns:
    interpolated_columns.append(p + '_interpolated')
    df_conductivity[p + '_interpolated'] = np.nan

# Iterate through inversions and interpolate them onto a new set of intervals     
for idx in df_inversions.site_id.unique():
    # Define the new intervals based on the conductivity intervals
    mask  = df_conductivity['SNMR_id']==idx
    new_intervals = df_conductivity[mask][['Depth_from', 'Depth_to']]
    # Extract the inversion data for the site
    mask = df_inversions['site_id']==idx
    df_inv = df_inversions[mask]
    # Interpolate and add the interpolated parameter columns to the dataframe
    df_conductivity.at[new_intervals.index,
                       interpolated_columns] = spatial_functions.interpolate_layered_model(df_inv,
                                                               parameter_columns,
                                                               interval_columns,
                                                               new_intervals)[interpolated_columns]


In [ ]:
# Remove columns with zeros for mobile or bound water

mask = (df_conductivity['Mobile_water_content_interpolated'] != 0)&(df_conductivity['Bound_water_content_interpolated'] != 0)

df_merged = df_conductivity[mask]


In [ ]:
df_conductivity

In [ ]:
# Estimate hydraulic conductivity using a nominal input into the SNMR equation
# This should really be done empirically

df_merged['k_estimate'] = SNMR_utils.tc_k_profile(df_merged['Mobile_water_content_interpolated'].values,
                                                 df_merged['Total_water_content_interpolated'].values,
                                                 N=2, C=4000)



In [ ]:
# Focussing on the mobile water content we will plot the results

# Note that this plot includes the unsaturated zone

plt.figure(figsize=(8, 8), dpi=80)
plt.scatter(np.log10(df_merged['conductivty'].values),
           df_merged['k_estimate'], 
           c = df_merged['Depth_to'])
plt.xlabel('log10 bulk AEM connductivity (S/m)')
plt.ylabel('hydraulic conductivity (m/day)')
plt.show()

In [24]:
df_merged

,Depth_from,Depth_to,SNMR_id,conductivity,conductivty,Mobile_water_content_interpolated,Bound_water_content_interpolated,Total_water_content_interpolated,k_estimate
0,0.000000,2.00,0,0.027278,NaN,0.000583,0.000583,0.001166,0.007256
1,2.000000,4.04,0,0.041689,NaN,0.000535,0.000910,0.001446,0.009688
2,4.040000,6.17,0,0.070287,NaN,0.001438,0.000398,0.001835,0.034867
3,6.170000,8.44,0,0.133710,NaN,0.000314,0.002641,0.002955,0.035330
4,8.440000,10.90,0,0.228633,NaN,0.000498,0.002217,0.002714,0.030500
5,10.900000,13.60,0,0.318797,NaN,0.001286,0.000659,0.001945,0.026896
6,13.600000,16.60,0,0.299774,NaN,0.000104,0.001683,0.001787,0.012821
8,19.969999,23.78,0,0.222891,NaN,0.000893,0.001171,0.002064,0.020956
9,23.780001,28.12,0,0.186917,NaN,0.001709,0.001297,0.003007,0.053423
10,28.120001,33.08,0,0.209898,NaN,0.001375,0.001026,0.002400,0.034295
